In [3]:
from ipycanvas import Canvas, hold_canvas
from Klasser.celle import *
from Klasser.busk import *
import random
from time import sleep

background_color = "#EEEEEE"

# visual size of tiles
tile_size = 15
visual_space = 2
rect_size = tile_size - visual_space

# amount of tiles in each direction
tiles_hori = 25
tiles_vert = 25

antal_celler = 10
start_energi = 10

antal_busker = 10

# Contains objects inhabiting the world
entities = []

# Create empty world, to contain references to cells
verd = []
for x in range(tiles_hori):
    row = []
    for y in range(tiles_vert):
        row.append(None)
    verd.append(row)

def get_nabolag(pos_x, pos_y, radius):
    nabolag = []
    nabolag_rad = []
    for y in range(pos_y - radius, pos_y + radius + 1):
        for x in range(pos_x - radius, pos_x + radius + 1):
            nabolag_rad.append(verd[x % tiles_hori][y % tiles_vert])
        nabolag.append(nabolag_rad)
        nabolag_rad = []
    return nabolag

def skap_celle(celle_1, celle_2):
    x = celle_1.pos_x + random.randint(-1, 1)
    y = celle_1.pos_y + random.randint(-1, 1)
    position_checks = 0
    while verd[x % tiles_hori][y % tiles_vert] != None:
        position_checks += 1
        if position_checks == 9:
            return
        x = celle_1.pos_x + random.randint(-1, 1)
        y = celle_1.pos_y + random.randint(-1, 1)
    energi = (celle_1.energi + celle_2.energi)/2
    sight_range = int((celle_1.sight_range + celle_2.sight_range)/2 + random.random() - 0.5)
    gen_mål_ønske = (celle_1.gen_mål_ønske + celle_2.gen_mål_ønske)/2 + (random.random() - 0.5)*0.1
    ny_celle = Celle(x % tiles_hori, y % tiles_vert, energi, sight_range, tiles_hori, tiles_vert, gen_mål_ønske, skap_celle)
    entities.append(ny_celle)
    verd[x % tiles_hori][y % tiles_vert] = len(entities) - 1

    # Cell needs to have a valid target at birth
    nabolag = get_nabolag(ny_celle.pos_x, ny_celle.pos_y, ny_celle.sight_range)
    ny_celle.finn_mål(nabolag, entities)
    return

# Populate world
celler_igjen = antal_celler
busker_igjen = antal_busker
while celler_igjen > 0 or busker_igjen > 0:
    x = random.randint(0, tiles_hori - 1)
    y = random.randint(0, tiles_vert - 1)
    while verd[x][y] != None:
        x = random.randint(0, tiles_hori - 1)
        y = random.randint(0, tiles_vert - 1)
    if celler_igjen > 0:
        sight_range = random.randint(1, 4)
        gen_mål_ønske = random.random()
        new_cell = Celle(x, y, start_energi, sight_range, tiles_hori, tiles_vert, gen_mål_ønske, skap_celle)
        entities.append(new_cell)
        verd[x][y] = len(entities) - 1
        celler_igjen -= 1
    else:
        new_busk = Busk(x, y, 0, 5)
        entities.append(new_busk)
        verd[x][y] = len(entities) - 1
        busker_igjen -= 1


# Canvas size
width = tile_size * tiles_hori
height = tile_size * tiles_vert

canvas = Canvas(height = height, width = width)
display(canvas)
vision_canvas = Canvas(height = height, width = width)
display(vision_canvas)

def draw_world(canvas):
    # Draw grid
    canvas.fill_style = background_color
    for x in range(tiles_hori):
        for y in range(tiles_vert):
            canvas.fill_rect(x * tile_size, y * tile_size, rect_size, rect_size)

def draw_entities(world, _canvas):
    with hold_canvas():
        x = 0
        for col in world:
            y = 0
            for tile in col:
                _canvas.fill_style = background_color
                if tile != None:
                    content = entities[tile]
                    if type(content) == Busk:
                        _canvas.fill_style = "green"
                    elif type(content) == Celle:
                        _canvas.fill_style = "blue"
                _canvas.fill_rect(x * tile_size, y * tile_size, rect_size, rect_size)
                y += 1
            x += 1

def rollover(list, index, radius):
    if index - radius < 0 or index + radius >= len(list) - 1:
        return list[index - radius:] + list[:(index + radius + 1) % len(list)]
    else:
        return list[index - radius : index + radius + 1]


def draw_nabolag(nabolag, entity):
    with hold_canvas():
        x = 0
        for col in nabolag:
            y = 0
            for tile in col:
                vision_canvas.fill_style = background_color
                if tile != None:
                    content = entities[tile]
                    if type(content) == Busk:
                        vision_canvas.fill_style = "green"
                    elif type(content) == Celle:
                        vision_canvas.fill_style = "blue"
                vision_canvas.fill_rect(y * tile_size, x * tile_size, rect_size, rect_size)
                y += 1
            x += 1
        vision_canvas.fill_style = "tomato"
        vision_canvas.fill_rect(entity.mål_x * tile_size, entity.mål_y * tile_size, rect_size, rect_size)

#Løkke som repeterast til programmet er ferdig
while True:
    for i in range(len(entities)):
        entity = entities[i]
        if type(entity) == Busk:
            entity.veks()
        elif type(entity) == Celle:
            nabolag = get_nabolag(entity.pos_x, entity.pos_y, entity.sight_range)
            entity.finn_mål(nabolag, entities)
            #draw_nabolag(nabolag, entity)
            beveg_x, beveg_y, dir_x, dir_y = entity.beveg()
            # Move the cell towards its target if there isnt anything in the way
            move_target = verd[beveg_x % tiles_hori][beveg_y % tiles_vert]
            if move_target == None:
                verd[entity.pos_x][entity.pos_y] = None
                entity.update_pos(beveg_x % tiles_hori, beveg_y % tiles_vert, dir_x, dir_y)
                verd[beveg_x % tiles_hori][beveg_y % tiles_vert] = i
            elif move_target == i:
                pass
            else:
                entity.har_mål = False

    # Draw everything
    with hold_canvas():
        draw_world(canvas)
        for entity in entities:
            if type(entity) == Celle and not entity.mål_y:
                print(entity.pos_x, entity.pos_y)
                print(entity.mål_x, entity.mål_y)
                pass

            entity.tegn(canvas, tile_size, rect_size)
            if type(entity) == Celle:
                nabolag = get_nabolag(entity.pos_x, entity.pos_y, entity.sight_range)
                #draw_nabolag(nabolag, entity)
    
    sleep(0.3)
    #input("press enter to continue")

Canvas(height=375, width=375)

Canvas(height=375, width=375)

11 18
1 0
11 18
2 0
11 19
1 0
12 18
3 0
11 19
2 0
11 17
0 0
11 20
1 0
bær 0
energi 11.0
19 18
4 0
21 18
2 0
20 18
3 0
11 17
0 0
11 20
2 0
12 17
0 0
9 20
0 0
10 19
0 0
bær 0
energi 11
bær 0
energi 11.0
11 17
0 0
10 19
0 0
11 21
0 0
14 18
0 0
11 17
0 0
12 18
0 0
10 19
0 0
14 18
0 0
10 21
0 0
12 21
0 0
8 21
0 0
11 17
0 0
12 18
0 0
10 19
0 0
14 18
0 0
12 21
0 0
7 19
2 0
14 21
0 0
9 21
0 0
12 22
0 0
bær 0
energi 11
21 18
2 0
11 17
0 0
12 18
0 0
11 23
0 0
12 21
0 0
7 19
2 0
14 21
0 0
12 22
0 0
23 18
0 0
12 15
0 0
13 22
0 0
10 24
0 0
11 24
0 0
14 18
0 0
9 23
0 0
7 22
0 0
bær 1
energi 11
bær 0
energi 11.0
10 18
2 0
19 18
4 0
11 17
0 0
9 24
0 0
11 23
0 0
14 19
0 0
10 23
0 0
12 21
0 0
9 22
0 0
12 22
0 0
12 15
0 0
14 18
0 0
9 23
0 0
12 16
0 0
10 19
0 0
14 22
0 0
9 0
0 0
11 20
0 0
15 18
0 0
15 3
1 0
24 16
0 0
8 22
0 0
16 2
1 0
9 21
0 0
bær 1
energi 11
bær 0
energi 12
bær 0
energi 12.0
bær 0
energi 12.0
10 18
2 0
11 17
0 0
12 19
0 0
9 24
0 0
10 21
0 0
14 19
0 0
12 21
0 0
9 22
0 0
12 22
0 0
12 15
0 

KeyboardInterrupt: 